# IHLT Project


In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import csv
import nltk

from utils.preprocessor import Preprocessor
from features.dummy_features import add_dummy_features
from features.statistical_features import add_POS_statistics, add_synset_statistics, add_synset_statistics_ext, add_lemma_statistics

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor


import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet_ic')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kacpe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Kacpe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kacpe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kacpe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kacpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\Kacpe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet_ic.zi

True

## Load data

In [2]:
TRAIN_PATH = '../data/train/01_raw/'
TRAIN_GS_PATH = '../data/train/scores/'
TEST_PATH = '../data/test/01_raw/'
TEST_GS_PATH = '../data/test/scores/'

def load_data(path_f, path_gs, files):
    # Read first file
    dt = pd.read_csv(path_f + 'STS.input.' + files[0] + '.txt', sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['s1', 's2'])
    dt['gs'] = pd.read_csv(path_gs + 'STS.gs.' + files[0] + '.txt', sep='\t', header=None, names=['gs'])
    # Concatenate the rest of files
    for f in files[1:]:
        dt2 = pd.read_csv(path_f + 'STS.input.' + f + '.txt', sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['s1', 's2'])
        dt2['gs'] = pd.read_csv(path_gs + 'STS.gs.' + f + '.txt', sep='\t', header=None, names=['gs'])
        dt = pd.concat([dt, dt2], ignore_index=True)
    return dt

## Create models

In [3]:
def train_NN(df, input, output):
    X = df[input]
    y = df[output]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
    model = Sequential()
    # model.add(Dense(10, input_dim=len(input), activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))               
    model.add(Dense(5, activation='relu'))               
    model.add(Dense(1))                                   
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, verbose=0)
    return history, model

def train_MLP(df, input, output):
    X = df[input]
    y = df[output]
    model = MLPRegressor(
        hidden_layer_sizes=(5, 5, 5), 
        max_iter=500, 
        random_state=42, 
        early_stopping=True, 
        validation_fraction=0.1,  
        verbose=True
    )
    history = model.fit(X, y)
    return history, model

def train_random_forest(df, input, output):
    X = df[input]
    y = df[output]

    # Definim i entrenem el model Random Forest
    model = RandomForestRegressor(n_estimators=50)
    model.fit(X, y)
    return model

def train_svr(df, input, output, kernel='rbf', C=1.0, epsilon=0.1):
    X = df[input]
    y = df[output]
    
    model = SVR(kernel=kernel, C=C, epsilon=epsilon)
    model.fit(X, y)
    
    return model

def train_svr_with_grid_search(df, input, output):
    X = df[input]
    y = df[output]
    
    model = SVR()
    
    param_grid = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto'],
        'degree': [2, 3, 4] 
    }
    
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=10)
    grid_search.fit(X, y)
    
    best_params = grid_search.best_params_
    print("Best params:", best_params)
    
    best_model = grid_search.best_estimator_
    
    return best_model

In [4]:
def plot_history(history):
    import matplotlib.pyplot as plt
    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('model mae')
    plt.ylabel('mae')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

## Basic pipeline

### Compute features

In [15]:
# Create the desired features
def add_features(dt):
    # add_POS_statistics(dt)
    # add_synset_statistics_ext(dt)
    add_lemma_statistics(dt)

# Load train data
print('Loading train data')
all_train_files = ['SMTeuroparl', 'MSRvid', 'MSRpar']
df = load_data(TRAIN_PATH, TRAIN_GS_PATH, all_train_files)
# Add features to the train data
add_features(df)
# Print all columns of resulting df
# print(df.columns)

# Load test data
print('Loading test data')
all_test_files = ['SMTeuroparl', 'MSRvid', 'MSRpar', 'surprise.OnWN', 'surprise.SMTnews']
df_test = load_data(TEST_PATH, TEST_GS_PATH, all_test_files)
# Add the features to the test data
add_features(df_test)

print('Train and test datasets ready')


Loading train data
Loading test data
Train and test datasets ready


In [16]:
# Features to use in the model
features = [
            # 's1_n_words', 's1_n_verbs_tot', 's1_n_verbs_pres', 's1_n_verbs_past', 's1_n_nouns', 's1_n_adjectives', 's1_n_adverbs', 
            # # 's2_n_words', 's2_n_verbs_tot', 's2_n_verbs_pres', 's2_n_verbs_past', 's2_n_nouns', 's2_n_adjectives', 's2_n_adverbs', 
            # 'dif_n_words', 'dif_n_verbs_tot', 'dif_n_verbs_pres', 'dif_n_verbs_past', 'dif_n_nouns', 'dif_n_adjectives', 'dif_n_adverbs', 
            
            # 'all_all_shared_synsets_count', 'all_all_shared_synsets_ratio', 'all_all_avg_synset_similarity', 'all_all_max_synset_similarity',
            # 'all_verb_shared_synsets_count', 'all_verb_shared_synsets_ratio', 'all_verb_avg_synset_similarity', 'all_verb_max_synset_similarity',
            # 'all_noun_shared_synsets_count', 'all_noun_shared_synsets_ratio', 'all_noun_avg_synset_similarity', 'all_noun_max_synset_similarity',
            # 'all_adj_shared_synsets_count', 'all_adj_shared_synsets_ratio', 'all_adj_avg_synset_similarity', 'all_adj_max_synset_similarity',
            # 'all_adv_shared_synsets_count', 'all_adv_shared_synsets_ratio', 'all_adv_avg_synset_similarity', 'all_adv_max_synset_similarity',

            # 'best_all_shared_synsets_count', 'best_all_shared_synsets_ratio', 'best_all_avg_synset_similarity', 'best_all_max_synset_similarity',
            # 'best_verb_shared_synsets_count', 'best_verb_shared_synsets_ratio', 'best_verb_avg_synset_similarity', 'best_verb_max_synset_similarity',
            # 'best_noun_shared_synsets_count', 'best_noun_shared_synsets_ratio', 'best_noun_avg_synset_similarity', 'best_noun_max_synset_similarity',
            # 'best_adj_shared_synsets_count', 'best_adj_shared_synsets_ratio', 'best_adj_avg_synset_similarity', 'best_adj_max_synset_similarity',
            # 'best_adv_shared_synsets_count', 'best_adv_shared_synsets_ratio', 'best_adv_avg_synset_similarity', 'best_adv_max_synset_similarity',

            'lemma_diversity', 'shared_lemmas_ratio', 'avg_lemma_similarity', 'max_lemma_similarity'
            ]

# Train a NN
hist, model = train_NN(df, features, 'gs')
# plot_history(hist)

# Train a MLP
# hist, model = train_MLP(df, features, 'gs')

# Train a Random Forest
# model = train_random_forest(df, features, 'gs')

# Train a Support vector regression
# model = train_svr(df, features, 'gs')

# Find best svr
# model = train_svr_with_grid_search(df, features, 'gs')

## Compute correlation of the model

In [17]:
from scipy.stats import pearsonr

# Fill column of the dataset with the predictions of the model
df_test['predicted'] = model.predict(df_test[features])

# Compute the Pearson correlation between the predictions and the gold standard
corr = pearsonr(df_test['gs'], df_test['predicted'])[0]
print('Pearson correlation:', corr)


98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Pearson correlation: 0.5462109997596549


In [ ]:
N_ITERS = 10
nn_p = 0
rf_p = 0

for i in range(N_ITERS):
    # print('Iteration:', i)
    # hist, model = train_NN(df, features, 'gs')
    # df_test['predicted'] = model.predict(df_test[features])
    # corr = pearsonr(df_test['gs'], df_test['predicted'])[0]
    # print('NN Pearson correlation:', corr)
    # nn_p += corr

    model = train_random_forest(df, features, 'gs')
    df_test['predicted'] = model.predict(df_test[features])
    corr = pearsonr(df_test['gs'], df_test['predicted'])[0]
    print('RF Pearson correlation:', corr)
    rf_p += corr

# print('NN Pearson correlation:', nn_p/N_ITERS)
print('RF Pearson correlation:', rf_p/N_ITERS)

RF Pearson correlation: 0.6981405928635337
RF Pearson correlation: 0.7016522605995392
RF Pearson correlation: 0.6937648052383595
RF Pearson correlation: 0.6915285461003057
RF Pearson correlation: 0.6939374465510104
RF Pearson correlation: 0.6968994286868317
RF Pearson correlation: 0.7035726529316195
RF Pearson correlation: 0.7018517155577366
RF Pearson correlation: 0.6952235142668917
RF Pearson correlation: 0.6967814757949129
NN Pearson correlation: 0.0
RF Pearson correlation: 0.6973352438590741
